In [14]:
from sklearn.datasets import make_swiss_roll
import numpy as np

In [2]:
X, y = make_swiss_roll()

In [32]:
X_centered = X - X.mean(axis=0)
U, s, Vt = np.linalg.svd(X_centered)

c1 = Vt.T[:, 0]
c2 = Vt.T[:, 1]

In [33]:
print(c1)
print(c2)

[-0.59922414 -0.1672811  -0.78290961]
[ 0.67255046  0.42530788 -0.60563115]


In [34]:
W2 = Vt.T[:, :2]
X2D = X_centered.dot(W2)

In [ ]:
# Principal Component Analysis:


In [31]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X2D = pca.fit_transform(X)

In [12]:
print('Original dimensions:', X[0])
print('2D:', X2D[0])

Original dimensions: [9.56561229 9.60653783 9.29435037]
2D: [11.52560562  1.13562523]


In [37]:
pca.explained_variance_ratio_

array([0.41443121, 0.31539339])

In [43]:
# Choosing the Right Number of Dimensions
pca = PCA()
pca.fit(X)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1

In [45]:
# PCA for Compression

pca = PCA(n_components = 154)
X_reduced = pca.fit_transform(X_train)
X_recovered = pca.inverse_transform(X_reduced)

In [16]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')

X_train, y_train = mnist["data"], mnist["target"]

# fetch_mldata was deprecated in v0.20
print(X_train)
print(y_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
['5' '0' '4' ... '4' '5' '6']


In [17]:
from sklearn.decomposition import IncrementalPCA

n_batches = 100
inc_pca = IncrementalPCA(n_components=154)

for X_batch in np.array_split(X_train, n_batches):
    inc_pca.partial_fit(X_batch)
    
X_reduced = inc_pca.transform(X_train)

In [23]:
# I don't think this is right
print(len(X_reduced[0])) ## 154
print(len(X_train[0]))   ## 784

154
784


In [24]:
# Example using np.memmap => won't run without external file

X_mm = np.memmap(filename, dtype="float32", mode="readonly", shape=(m, n))
batch_size = m // n_batches
inc_pca = IncrementalPCA(n_components=154, batch_size=batch_size)
inc_pca.fit(X_mm)

NameError: name 'filename' is not defined

In [25]:
# Randomized PCA:
#  - Stochastic algorithm that finds approximation of first d principal components
#  - Significantly faster than other algorithms if d is much smaller than n

In [26]:
rnd_pca = PCA(n_components=154, svd_solver="randomized")
X_reduced = rnd_pca.fit_transform(X_train)

In [27]:
len(X_reduced[0])

154

In [52]:
# Kernel PCA:
#  - Applies the kernel trick to PCA
#    - Makes it possible to perform complex nonlinear projections for dimensionality reduction
#    - Is often good at preserving clusters of instances after projection, 
#      or sometimes unrolling datasets that lie close to a twisted manifold

In [48]:
from sklearn.decomposition import KernelPCA

rbf_pca = KernelPCA(n_components = 2, kernel="rbf", gamma=0.04)
X_reduced = rbf_pca.fit_transform(X)

In [50]:
print('X:', X)
print('X_reduced:', X_reduced)

X: [[  9.56561229   9.60653783   9.29435037]
 [  9.17962368  17.86024339   9.73603468]
 [  3.63687111  16.27361652   6.37012886]
 [ -8.99692117  10.4283866   -3.99068708]
 [  1.98603501  18.84781087 -10.99634834]
 [  0.59245301  19.40260127  -4.79879271]
 [  6.09979333   2.71868632  -0.80256408]
 [  4.99117312  18.13469083  12.82945513]
 [ 11.79667249  13.21833715  -3.42496738]
 [  4.06187479   4.83865712  -3.76165458]
 [  0.50571782  19.96296288  -4.79093954]
 [ 12.54511438   7.1472712   -0.2382997 ]
 [  3.75743805  16.07810275  13.34375268]
 [ -3.10371986  12.4277686    7.63330034]
 [  5.80933885  20.68429125  -9.95251061]
 [ -2.88564696  19.54608717   7.68936538]
 [  2.88596173   1.87097079  -4.43283204]
 [  4.04051982   0.26220694   6.03671701]
 [ 11.49348173  11.54096795   6.20482456]
 [ -7.34444383   8.79026932  -7.06476576]
 [ -7.98831909  11.04395985  -6.14891285]
 [ 12.57892318   3.12774009   0.17284578]
 [  1.93646986   4.22276763  13.86379868]
 [ -3.97576258   9.89023042   7

In [54]:
# Selecting a Kernel and Tuning Hyperparameters
# kPCA is unsupervised, so there is no obvious performance measure to help select best keernel and hyperparameters

# by creating a pipeline, GridSearch will find the best kernel and gamma values for kPCA 
# in order to get the best classification accuracy at the end of the pipeline

In [57]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ("kpca", KernelPCA(n_components=2)),
    ("log_reg", LogisticRegression())
])

param_grid = [{
    "kpca_gamma": np.linspace(0.03, 0.05, 10),
    "kpca_kernel": ["rbf", "sigmoid"]
}]

grid_search = GridSearchCV(clf, param_grid, cv=3)

grid_search.fit(X, y)

/Users/austinkovach/Desktop/Odin/machine-learning-geron/venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/austinkovach/Desktop/Odin/machine-learning-geron/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:530: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: Unknown label type: 'continuous'

In [59]:
# Can also find kernel and hyperparameters with lowest reconstruction error
# Reconstruction ith kPCA is harder than linear PCA
# Because of the kernel trick, training set has been mapped to infinite-dimensional feature space
# This means that the reconstruction would lie in feature space, not the original space

# Best approximation is to find a point in original space that would map close to the reconstructed point
#   - This reconstructed point is the "pre-image"
# Then measure squared distance from pre-image to original instance
# Select kernel and hyperparameters than minimize reconstruction pre-image error

# One solution to perform this reconstruction is a supervised regression model
#  - projected instances as the training set
#  - original instances as targets
# Scikit Learn will create this automatically with fit_inverse_transform=True

In [58]:
rbf_pca = KernelPCA(n_components = 2, kernel="rbf", gamma=0.0433, fit_inverse_transform=True)

X_reduced = rbf_pca.fit_transform(X)
X_preimage = rbf_pca.inverse_transform(X_reduced)

In [61]:
from sklearn.metrics import mean_squared_error
mean_squared_error(X, X_preimage)

39.810754879335285

In [62]:
# LLE:
#  Locally Linear Embedding
#  non-linear dimensionality reduction (NLDR) technique

# Good at unrolling twisted manifolds where there is not much noise

# Works by finding k closest neighbors for a training instance,
# then reconstructs the training instance as a linear function of these neighbors

In [63]:
from sklearn.manifold import LocallyLinearEmbedding

lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10)
X_reduced = lle.fit_transform(X)

In [65]:
# Other dimensionality reduction techniques:

# Multidimensional Scaling (MDS)
# Isomap
# t-Distributed Stochastic Neighbor Embedding (t-SNE)
# Linear Discriminant Analysis (LDA)